In [ ]:
import random
import numpy as np
import pandas as pd
import math

def simulateDailyDemand(p):
    x = [0]*6
    for i in range(6):
        x[i] = np.random.binomial(1,p/(6))
    
    y = sum(x)
    return(y)

In [ ]:
def simulateDemand(): #Simulates demand for a week
    # all the branches are considered as 1 to 24 as given order
    demAlternator = [0]*24 #saves demand for alternator across all the branches
    demGearBox = [0]*24 #saves demand for Gear boxes across all the branches
    demSelfStarter = [0]*24 #saves demand for self starters across all the branches
    

    mean_demand = pd.read_csv('demand_data.csv')
    
    mean_demAlternator = np.array(mean_demand['1'])
    mean_demGearBox = np.array(mean_demand['2'])
    mean_demSelfStarter = np.array(mean_demand['3'])
    
    for i in range(24):
        if(mean_demAlternator[i-1]<1):
            demAlternator[i-1] = np.random.binomial(1,mean_demAlternator[i-1])
        else:
            demAlternator[i-1] = simulateDailyDemand(mean_demAlternator[i-1])
            
        
    for i in range(24):
        if(mean_demGearBox[i-1]<1):
            demGearBox[i-1] = np.random.binomial(1,mean_demGearBox[i-1])
        else:
            demGearBox[i-1] = simulateDailyDemand(mean_demGearBox[i-1])
            
    for i in range(24):
        if(mean_demSelfStarter[i-1]<1):
            demSelfStarter[i-1] = np.random.binomial(1,mean_demSelfStarter[i-1])
        else:
            demSelfStarter[i-1] = simulateDailyDemand(mean_demSelfStarter[i-1])
            
    demAlternator = np.array(demAlternator)
    demGearBox = np.array(demGearBox)
    demSelfStarter = np.array(demSelfStarter)
    return(demAlternator,demGearBox,demSelfStarter)

In [ ]:
def SilmulateDemandDiv(demAltB,demGbB,demSsB):
    demAlt1 = np.sum(demAltB[0:9])
    demAlt2 = np.sum(demAltB[10:18])
    demAlt3 = np.sum(demAltB[19:23])
    demAltD = np.array([demAlt1,demAlt2,demAlt3])
    
    demGb1 = np.sum(demGbB[0:9])
    demGb2 = np.sum(demGbB[10:18])
    demGb3 = np.sum(demGbB[19:23])
    demGbD = np.array([demGb1,demGb2,demGb3])
    
    demSs1 = np.sum(demSsB[0:9])
    demSs2 = np.sum(demSsB[10:18])
    demSs3 = np.sum(demSsB[19:23])
    demSsD = np.array([demSs1,demSs2,demSs3])
    
    return(demAltD,demGbD,demSsD)

In [ ]:
#Returns Allocation from RCU to branches/divisions
def allocation(A,cap,noOfStaions):
    wA = A/np.sum(A)
    rem = cap - np.sum((np.floor(wA*cap)))
    b= np.floor(wA*cap) #initial allocation
    while rem>0:
        for idx in range(noOfStaions):
             if wA[idx]!=0:
                b[idx] +=1
                rem-=1
                if rem<=0:
                    break
    return b

In [ ]:
def operationsOld(T,float_):
    a=0
    b=0
    c=0
    MHAlt = 32*6
    MHGb = 56*6
    MHSs = 48*6
    
    STAlt = 8
    STGb = 12
    STSs = 8
    
    #may be T+2 is needed
#     invAltB=[[0]*(T+1)]*24
    invAltB = np.zeros((24,T+1))
    
    invGbB=np.zeros((24,T+1))
    invSsB=np.zeros((24,T+1))
    
    UnfulfiledDemAltB=np.zeros((24,T+1))
    UnfulfiledDemGbB=np.zeros((24,T+1))
    UnfulfiledDemSsB=np.zeros((24,T+1))

    initialFloaData = pd.read_csv('10_float.csv')
    initialFloat = np.array(initialFloaData['1'])
    initialFloat = np.ceil(float_*initialFloat)
#     print(initialFloat)

    UPinvAltRCU=np.zeros(T+1)
    UPinvGbRCU=np.zeros(T+1)
    UPinvSsRCU=np.zeros(T+1)
    
    PinvAltRCU=np.zeros(T+1)
    PinvGbRCU=np.zeros(T+1)
    PinvSsRCU=np.zeros(T+1)
    
    OdrAlt = np.zeros((24,T+1))
    OdrGb = np.zeros((24,T+1))
    OdrSs = np.zeros((24,T+1))
    
    #ith list represents inventory at ith branch
    for i in range(24):
        #invAltB.append([initialFloat[i-1]])
        invAltB[i][0] = initialFloat[i]
#         print(initialFloat[i],invAltB[i][0])
        #UnfulfiledDemAltB[i][0]=0
        #UPinvAltRCU.append(0)
        #PinvAltRCU.append(0)
        #OdrAlt[i][0]=0
#         print(invAltB[i][0]) 
        
    for j in range(24):
        #invGbB.append([initialFloat[i-1]])
        invGbB[j][0] = initialFloat[j]
        #UnfulfiledDemGbB[i-1][0]=0#
        #UPinvGbRCU.append(0)
        #PinvGbRCU.append(0)
        #OdrGb[i-1][0]=0#
        
    for k in range(24):
        invSsB[k][0] = initialFloat[k]
#         print(initialFloat[i],invSsB[i][0])
        #UnfulfiledDemSsB[i-1][0]=0#
        #UPinvSsRCU.append(0)
        #PinvSsRCU.append(0)
        #OdrSs[i-1][0]=0#
    #print(invAltB)        
    for t in range(T):
        (demAlt_t,demGB_t,demSS_t) = simulateDemand()
        #print(demAlt_t)
        a = a + sum(demAlt_t)
        b = b + sum(demGB_t)
        c = c + sum(demSS_t)
        for i in range(24):#Look into this, there is an error
#             print(invAltB[i][0])
            invAltB[i][t+1]=max((invAltB[i][t]-demAlt_t[i]),0)
            invGbB[i][t+1]=max((invGbB[i][t]-demGB_t[i]),0)
            invSsB[i][t+1]=max((invSsB[i][t]-demSS_t[i]),0)
#             print(demAlt_t[i],invAltB[i][t],invAltB[i][t+1])
            
            #Look into this, there is an error
            OdrAlt[i][t] = OdrAlt[i][t] + demAlt_t[i]
            OdrGb[i][t] = OdrGb[i][t] + demGB_t[i]
            OdrSs[i][t] = OdrSs[i][t] + demSS_t[i]
            
            UnfulfiledDemAltB[i][t]= max((-invAltB[i][t]+demAlt_t[i]),0)#Ok till here
            UnfulfiledDemGbB[i][t] = max((-invGbB[i][t]+demGB_t[i]),0)
            UnfulfiledDemSsB[i][t] = max ((-invSsB[i][t]+demSS_t[i]),0)
            
        #This is where things went wrong, update later with t+1    
#         UPinvAltRCU[t] = max(UPinvAltRCU[t] - math.floor(MHAlt/STAlt),0 ) +sum(demAlt_t)
#         UPinvGbRCU[t] = max(UPinvGbRCU[t] - math.floor(MHGb/STGb),0) +sum(demGB_t)
#         UPinvSsRCU[t] = max(UPinvSsRCU[t] - math.floor(MHGb/STGb),0) +sum(demSS_t)
        
        #At the begining of the week
        UPinvAltRCU[t] = UPinvAltRCU[t] +sum(demAlt_t)
        UPinvGbRCU[t] = UPinvGbRCU[t] +sum(demGB_t)
        UPinvSsRCU[t] = UPinvSsRCU[t] +sum(demSS_t)
        
        PinvAltRCU[t] = PinvAltRCU[t] + min(math.floor(MHAlt/STAlt), UPinvAltRCU[t])#Say has been processed by the end of the that week
        PinvGbRCU[t] = PinvGbRCU[t] + min(math.floor(MHGb/STGb), UPinvGbRCU[t])
        PinvSsRCU[t] = PinvSsRCU[t] + min(math.floor(MHGb/STGb),UPinvSsRCU[t])
        
        UPinvAltRCU[t+1] = UPinvAltRCU[t] - min(math.floor(MHAlt/STAlt), UPinvAltRCU[t])
        UPinvGbRCU[t+1] = UPinvGbRCU[t] - min(math.floor(MHGb/STGb), UPinvGbRCU[t])
        UPinvSsRCU[t+1] = UPinvSsRCU[t] - min(math.floor(MHGb/STGb),UPinvSsRCU[t])
        
        allocationAlt = allocation(demAlt_t,math.floor(MHAlt/STAlt),24)
        allocationGb = allocation(demGB_t,math.floor(MHGb/STGb),24)
        allocationSs = allocation(demSS_t,math.floor(MHGb/STGb),24)
        
        if(PinvAltRCU[t]>0):
            for i in range(24):
                invAltB[i][t+1] = invAltB[i][t+1] + allocationAlt[i]
            PinvAltRCU[t+1] = PinvAltRCU[t] - np.sum(allocationAlt)
            
        if(PinvGbRCU[t]>0):
            for i in range(24):
                invGbB[i][t+1] = invGbB[i][t+1] + allocationGb[i]
            PinvGbRCU[t+1] = PinvGbRCU[t] - np.sum(allocationGb)
            
        if(PinvSsRCU[t]>0):
            for i in range(24):
                invSsB[i][t+1] = invSsB[i][t+1] + allocationSs[i]
            PinvSsRCU[t+1] = PinvSsRCU[t] - np.sum(allocationSs)
            
            
        for j in range(24):
            OdrAlt[j][t+1] = OdrAlt[j][t] - allocationAlt[i]
            OdrGb[j][t+1] = OdrGb[j][t] - allocationGb[i]
            OdrSs[j][t+1] = OdrSs[j][t] - allocationSs[i]
            
    total_unfulfiled_dem = sum(sum(np.array(UnfulfiledDemAltB)))+sum(sum(np.array(UnfulfiledDemGbB)))+sum(sum(np.array(UnfulfiledDemSsB)))
    total_dem = a + b + c
    #print(total_dem)
    SL = 1 - total_unfulfiled_dem/total_dem
    #print(invAltB)
#     print(SL)
    #return(SL,invAltB,UnfulfiledDemAltB)
    return(SL)

In [ ]:
avgSLOld = 0
for i in range(100):
    a = operationsOld(26,0.1)
    avgSLOld = avgSLOld + float(a)    
avgSLOld = avgSLOld/100

In [ ]:
avgSLOld

In [ ]:
def operationsNew(T,float_):
    a=0
    b=0
    c=0
    MHAlt = 32*6
    MHGb = 56*6
    MHSs = 48*6
    
    STAlt = 8
    STGb = 12
    STSs = 8
    
    #may be T+2 is needed
#     invAltB=[[0]*(T+1)]*24
    invAltB = np.zeros((3,T+1))
    
    invGbB=np.zeros((3,T+1))
    invSsB=np.zeros((3,T+1))
    
    UnfulfiledDemAltB=np.zeros((3,T+1))
    UnfulfiledDemGbB=np.zeros((3,T+1))
    UnfulfiledDemSsB=np.zeros((3,T+1))
    
    
    initialFloaData = pd.read_csv('10_float_div.csv')
    initialFloat = np.array(initialFloaData['1'])
    initialFloat = np.ceil(float_*initialFloat)
    
    
    UPinvAltRCU=np.zeros(T+1)
    UPinvGbRCU=np.zeros(T+1)
    UPinvSsRCU=np.zeros(T+1)
    
    PinvAltRCU=np.zeros(T+1)
    PinvGbRCU=np.zeros(T+1)
    PinvSsRCU=np.zeros(T+1)
    
    OdrAlt = np.zeros((3,T+1))
    OdrGb = np.zeros((3,T+1))
    OdrSs = np.zeros((3,T+1))
    
    #ith list represents inventory at ith branch
    for i in range(3):
        #invAltB.append([initialFloat[i-1]])
        invAltB[i][0] = initialFloat[i]
#         print(initialFloat[i],invAltB[i][0])
        #UnfulfiledDemAltB[i][0]=0
        #UPinvAltRCU.append(0)
        #PinvAltRCU.append(0)
        #OdrAlt[i][0]=0
#         print(invAltB[i][0]) 
        
    for j in range(3):
        #invGbB.append([initialFloat[i-1]])
        invGbB[j][0] = initialFloat[j]
        #UnfulfiledDemGbB[i-1][0]=0#
        #UPinvGbRCU.append(0)
        #PinvGbRCU.append(0)
        #OdrGb[i-1][0]=0#
        
    for k in range(3):
        invSsB[k][0] = initialFloat[k]
#         print(initialFloat[i],invSsB[i][0])
        #UnfulfiledDemSsB[i-1][0]=0#
        #UPinvSsRCU.append(0)
        #PinvSsRCU.append(0)
        #OdrSs[i-1][0]=0#
    #print(invAltB)        
    for t in range(T):
        (demAlt,demGb,demSs) = simulateDemand()
        (demAlt_t,demGB_t,demSS_t) = SilmulateDemandDiv(demAlt,demGb,demSs)
        #print(demAlt_t,demGB_t,demSS_t)
        #print(demAlt_t)
        a = a + np.sum(demAlt_t)
        b = b + np.sum(demGB_t)
        c = c + np.sum(demSS_t)
        for i in range(3):#Look into this, there is an error
#             print(invAltB[i][0])
            #print(invAltB[i][t+1])
            invAltB[i][t+1]=max((invAltB[i][t]-demAlt_t[i]),0)
            #print(invAltB[i][t+1])
            invGbB[i][t+1]=max((invGbB[i][t]-demGB_t[i]),0)
            invSsB[i][t+1]=max((invSsB[i][t]-demSS_t[i]),0)
#             print(demAlt_t[i],invAltB[i][t],invAltB[i][t+1])
            
            #Look into this, there is an error
            OdrAlt[i][t] = OdrAlt[i][t] + demAlt_t[i]
            OdrGb[i][t] = OdrGb[i][t] + demGB_t[i]
            OdrSs[i][t] = OdrSs[i][t] + demSS_t[i]
            
            UnfulfiledDemAltB[i][t]= max((-invAltB[i][t]+demAlt_t[i]),0)#Ok till here
            UnfulfiledDemGbB[i][t] = max((-invGbB[i][t]+demGB_t[i]),0)
            UnfulfiledDemSsB[i][t] = max ((-invSsB[i][t]+demSS_t[i]),0)
            
        #This is where things went wrong, update later with t+1    
#         UPinvAltRCU[t] = max(UPinvAltRCU[t] - math.floor(MHAlt/STAlt),0 ) +sum(demAlt_t)
#         UPinvGbRCU[t] = max(UPinvGbRCU[t] - math.floor(MHGb/STGb),0) +sum(demGB_t)
#         UPinvSsRCU[t] = max(UPinvSsRCU[t] - math.floor(MHGb/STGb),0) +sum(demSS_t)
        
        #At the begining of the week
        UPinvAltRCU[t] = UPinvAltRCU[t] +sum(demAlt_t)
        UPinvGbRCU[t] = UPinvGbRCU[t] +sum(demGB_t)
        UPinvSsRCU[t] = UPinvSsRCU[t] +sum(demSS_t)
        
        PinvAltRCU[t] = PinvAltRCU[t] + min(math.floor(MHAlt/STAlt), UPinvAltRCU[t])#Say has been processed by the end of the that week
        PinvGbRCU[t] = PinvGbRCU[t] + min(math.floor(MHGb/STGb), UPinvGbRCU[t])
        PinvSsRCU[t] = PinvSsRCU[t] + min(math.floor(MHGb/STGb),UPinvSsRCU[t])
        
        UPinvAltRCU[t+1] = UPinvAltRCU[t] - min(math.floor(MHAlt/STAlt), UPinvAltRCU[t])
        UPinvGbRCU[t+1] = UPinvGbRCU[t] - min(math.floor(MHGb/STGb), UPinvGbRCU[t])
        UPinvSsRCU[t+1] = UPinvSsRCU[t] - min(math.floor(MHGb/STGb),UPinvSsRCU[t])
        
        allocationAlt = allocation(demAlt_t,math.floor(MHAlt/STAlt),3)
        allocationGb = allocation(demGB_t,math.floor(MHGb/STGb),3)
        allocationSs = allocation(demSS_t,math.floor(MHGb/STGb),3)
        
        if(PinvAltRCU[t]>0):
            for i in range(3):
                invAltB[i][t+1] = invAltB[i][t+1] + allocationAlt[i]
            PinvAltRCU[t+1] = PinvAltRCU[t] - np.sum(allocationAlt)
            
        if(PinvGbRCU[t]>0):
            for i in range(3):
                invGbB[i][t+1] = invGbB[i][t+1] + allocationGb[i]
            PinvGbRCU[t+1] = PinvGbRCU[t] - np.sum(allocationGb)
            
        if(PinvSsRCU[t]>0):
            for i in range(3):
                invSsB[i][t+1] = invSsB[i][t+1] + allocationSs[i]
            PinvSsRCU[t+1] = PinvSsRCU[t] - np.sum(allocationSs)
             
        for j in range(3):
            OdrAlt[j][t+1] = OdrAlt[j][t] - allocationAlt[i]
            OdrGb[j][t+1] = OdrGb[j][t] - allocationGb[i]
            OdrSs[j][t+1] = OdrSs[j][t] - allocationSs[i]
            
    total_unfulfiled_dem = sum(sum(np.array(UnfulfiledDemAltB)))+sum(sum(np.array(UnfulfiledDemGbB)))+sum(sum(np.array(UnfulfiledDemSsB)))
    total_dem = a + b + c
    #print(total_dem)
    SL = 1 - total_unfulfiled_dem/total_dem
    #print(invAltB)
#     print(SL)
    #return(SL,invAltB,UnfulfiledDemAltB)
    return(SL)
        

In [ ]:
avgSLNew = 0
for i in range(100):
    avgSLNew = avgSLNew + float(operationsNew(26,0.1))
avgSLNew = avgSLNew/100

In [ ]:
avgSLNew

In [ ]:
np.arange(0,10,1)

In [ ]:
#For graph and table generation
n_epoch = 20
old_sl = list()
new_sl = list()
n_div = 10

for floatsize in np.arange(0,n_div,1):
    avgSLOld = 0
    avgSLNew = 0
    print(floatsize)
    for i in range(n_epoch):
        avgSLOld = avgSLOld + float(operationsOld(26,floatsize/n_div + 0.000001))    
        avgSLNew = avgSLNew + float(operationsNew(26,floatsize/n_div + 0.000001))
    
    avgSLOld = avgSLOld/n_epoch
    avgSLNew  = avgSLNew/n_epoch
    old_sl.append(avgSLOld)
    new_sl.append(avgSLNew)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
np.arange(0,11,1)/10

In [ ]:
plt.plot(np.arange(1,11,1)/10,old_sl,label = "Old Policy")
plt.plot(np.arange(1,11,1)/10,new_sl,label = "New Policy")
plt.ylabel("Service Level")
plt.xlabel("Float Size in fraction of fleet")
plt.legend()
plt.show()

In [ ]:
for floatsize in np.arange(0,10,1):
    print(floatsize/10 + 0.000001)

In [ ]:
n_epoch = 20
old_sl = list()
new_sl = list()
n_div = 20

for floatsize in np.arange(0,n_div,1):
    avgSLOld = 0
    avgSLNew = 0
    print(floatsize)
    for i in range(n_epoch):
        avgSLOld = avgSLOld + float(operationsOld(26,floatsize/n_div + 0.00000001))    
        avgSLNew = avgSLNew + float(operationsNew(26,floatsize/n_div + 0.00000001))
    
    avgSLOld = avgSLOld/n_epoch
    avgSLNew  = avgSLNew/n_epoch
    old_sl.append(avgSLOld)
    new_sl.append(avgSLNew)

In [ ]:
old_sl

In [ ]:
new_sl